# K均值聚类

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True, threshold=16)
%matplotlib inline

## 机器学习实战

In [ ]:
data = np.loadtxt('testSet.txt')
data

In [ ]:
plt.scatter(data[:, 0], data[:, 1])

In [ ]:
class KMeans:
    def __init__(self, k):
        self.k = k  # 质心数目
    
    def init_(self, X):
        X_max = np.max(X, axis=0)
        X_min = np.min(X, axis=0)
        m0 = np.zeros((self.k, X.shape[1]))
        for i in range(self.k):
            m_ = np.random.rand(X.shape[1]) * (X_max - X_min) + X_min
            m0[i] = m_
        return m0
      
    def fit(self, X):
        n_samples, n_features = X.shape
        # 保存聚类的结果, (所属类编号, 样本到所属类中心距离)
        cluster = np.zeros((n_samples, 2)) 
        # 初始化 中心
        m = self.init_(X)

        while True:
            cluster_last = cluster.copy()
            # 计算样本到各中心点距离, 指派到最近的点
            for i in range(n_samples):
                distance = np.sum(np.square(X[i] - m), axis=1)
                min_index = np.argmin(distance)
                cluster[i] = (min_index, distance[min_index])
            
            # 计算新的中心
            # print(m)
            for l in range(self.k):
                m[l] = np.mean(X[cluster[:, 0] == l,:], axis=0)
            if np.all(cluster[:, 0] == cluster_last[:, 0]):
                break
        return m, cluster

In [ ]:
clf = KMeans(4)
m, cluster = clf.fit(data)

In [ ]:
plt.scatter(data[cluster[:, 0]==0, 0], data[cluster[:, 0]== 0, 1], marker='o')
plt.scatter(data[cluster[:, 0]==1, 0], data[cluster[:, 0]== 1, 1], marker='v')
plt.scatter(data[cluster[:, 0]==2, 0], data[cluster[:, 0]== 2, 1], marker='+')
plt.scatter(data[cluster[:, 0]==3, 0], data[cluster[:, 0]== 3, 1], marker='x')
plt.scatter(m[:, 0], m[:, 1], marker='s', s=80)

In [ ]:
SSE = np.sum(cluster[:, 1])
SSE

### 二分k-means算法
二分k-means算法是为了克服k-means算法收敛于局部最小值的问题,二分k-kmeans算法首先将所有点作为一个簇,然后将该簇一分为二.之后选择其中一个簇继续进行划分,选择哪一个簇进行划分取决于对其划分是否可以最大程度降低SSE值.上述基于SSE的划分过程不断重复,直到得到用户指定的簇数目为止

二分k-means的伪代码形式如下:
* 将所有点看成一个簇
* 当簇数目小于 k 时
* 对于每一个簇
    * 计算总误差
    * 在给定的簇上面进行 KMeans 聚类（k=2）
    * 计算将该簇一分为二之后的总误差
* 选择使得误差最小的那个簇进行划分操作

另一种做法是选择 SSE 最大的簇进行划分，直到簇数目达到用户指定的数目位置。

In [ ]:
# 二分 K-Means 聚类算法
class BinMeans:
    def __init__(self, k):
        self.k = k
          
    def choose_best_cluster(self, X, cluster, m):
        # 选择最优的簇进行分类, 
        max_diff = 0
        # 当前所有类别
        m_index = np.unique(cluster[:, 0]).astype(int).tolist()
        cluster_m = []
        for l in m_index:
            data = X[cluster[:, 0]==l, :]
            # 聚类之前的SSE
            sse_before = np.sum(cluster[cluster[:, 0]==l, 1])
            clf = KMeans(2)
            new_m, new_cluster = clf.fit(data)
            # 新的类的编号
            new_cluster[:, 0] += len(m)
            cluster_m.append((new_cluster, new_m))
            # 最大程度降低SSE的值
            diff = sse_before - np.sum(new_cluster[:, 1]) 
            if diff > max_diff:
                best_index = l
                max_diff = diff
        # 添加新的中心点
        m.extend(cluster_m[m_index.index(best_index)][1])
        j = 0
        # 更新每个样本的类别编号和距离中心的距离
        for i in range(len(cluster)):
            if cluster[i, 0] == best_index:
                cluster[i] = cluster_m[m_index.index(best_index)][0][j]
                j += 1
        return cluster, m
            
            
    def fit(self, X):
        n_samples, n_features = X.shape
        # 所有样本分到一个簇中
        m_ = np.mean(X, axis=0)
        m = [m_]
        cluster = np.zeros((n_samples, 2))
        cluster[:, 1] = np.sum(np.square(X - m[0]), axis=1)
        
        while len(np.unique(cluster[:, 0])) < self.k:
            cluster, m = self.choose_best_cluster(X, cluster, m)
            # 对sse最大的簇进行 K-Means聚类(k=2)
        return np.array(m), cluster

In [ ]:
clf = BinMeans(4)
m, cluster = clf.fit(data)

In [ ]:
m

In [ ]:
SSE = np.sum(cluster[:, 1])
SSE

In [ ]:
np.unique(cluster[:, 0])

In [ ]:
plt.scatter(data[cluster[:, 0]==3, 0], data[cluster[:, 0]== 3, 1], marker='o')
plt.scatter(data[cluster[:, 0]==4, 0], data[cluster[:, 0]== 4, 1], marker='v')
plt.scatter(data[cluster[:, 0]==5, 0], data[cluster[:, 0]== 5, 1], marker='+')
plt.scatter(data[cluster[:, 0]==6, 0], data[cluster[:, 0]== 6, 1], marker='x')
plt.scatter(m[3:, 0], m[3:, 1], marker='s', s=80)

In [ ]:
data = np.loadtxt('./testSet2.txt')
plt.scatter(data[:, 0], data[:, 1])

In [ ]:
clf_k = KMeans(k=3)
m, cluster = clf_k.fit(data)

In [ ]:

plt.scatter(data[cluster[:, 0]==0, 0], data[cluster[:, 0]== 0, 1], marker='o')
plt.scatter(data[cluster[:, 0]==1, 0], data[cluster[:, 0]== 1, 1], marker='v')
plt.scatter(data[cluster[:, 0]==2, 0], data[cluster[:, 0]== 2, 1], marker='s')
# plt.scatter(data[cluster[:, 0]==3, 0], data[cluster[:, 0]== 3, 1], marker='x')
plt.scatter(m[:, 0], m[:, 1], marker='+', s=100, c='k')

In [ ]:
clf_k = BinMeans(k=3)
m, cluster = clf_k.fit(data)

In [ ]:
np.unique(cluster[:, 0])

In [ ]:
plt.scatter(data[cluster[:, 0]==2, 0], data[cluster[:, 0]== 2, 1], marker='o')
plt.scatter(data[cluster[:, 0]==3, 0], data[cluster[:, 0]== 3, 1], marker='v')
plt.scatter(data[cluster[:, 0]==4, 0], data[cluster[:, 0]== 4, 1], marker='s')
# plt.scatter(data[cluster[:, 0]==3, 0], data[cluster[:, 0]== 3, 1], marker='x')
plt.scatter(m[2:, 0], m[2:, 1], marker='+', s=100, c='k')

## sklearn  KMeans

```
Signature:
make_blobs(
    n_samples=100,
    n_features=2,
    centers=None,
    cluster_std=1.0,
    center_box=(-10.0, 10.0),
    shuffle=True,
    random_state=None,
)
Docstring:
Generate isotropic Gaussian blobs for clustering.

Read more in the :ref:`User Guide <sample_generators>`.
```

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

blobs, classes = make_blobs(500, centers=3)

In [ ]:
color_map = np.array(['r', 'g', 'b']) 
plt.scatter(blobs[:, 0], blobs[:, 1], color=color_map[classes])

```
class sklearn.cluster.KMeans(n_clusters=8, init=’k-means++’, n_init=10,
                            max_iter=300, tol=0.0001, precompute_distances=’auto’,
                            verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm=’auto’)
```


In [ ]:
# 假设已知有3个中心
kmean = KMeans(n_clusters=3)
kmean.fit(blobs)

In [ ]:
classes

In [ ]:
# 每个样本点的预期标签, 由于 KMeans 不知道类别是什么, 与classes会不一致
kmean.labels_

In [ ]:
color_map = np.array(['r', 'g', 'b']) 
plt.scatter(blobs[:, 0], blobs[:, 1], color=color_map[classes])
# cluster 中心点坐标
plt.scatter(kmean.cluster_centers_[:, 0], kmean.cluster_centers_[:, 1], marker='*', s=250, color='k')

In [ ]:
kmean.predict([[5, -1], [-8, 3], [5, 5]])

In [ ]:
# 输出每个点到形心的距离
kmean.transform(blobs[:5])

### 优化形心数量
---
形心难以解释，并且也难以判断是否数量正确。理解你的数据是否是未分类的十分重要，因为这会直接影响我们可用的评估手段。

In [ ]:
blobs, classes = make_blobs(500, centers=3)
kmean = KMeans(n_clusters=3)
kmean.fit(blobs)

In [ ]:
color_map = np.array(['r', 'g', 'b']) 
plt.scatter(blobs[:, 0], blobs[:, 1], color=color_map[classes])

In [ ]:
cluster_labels = kmean.labels_
blobs[cluster_labels==2]
plt.scatter(blobs[cluster_labels==2][:, 0], blobs[cluster_labels==2][:, 1])
plt.scatter(blobs[cluster_labels==1][:, 0], blobs[cluster_labels==1][:, 1])
plt.scatter(blobs[cluster_labels==0][:, 0], blobs[cluster_labels==0][:, 1])

首先，我们查看**轮廓**（Silhouette）距离。轮廓距离是簇内不相似性、最近的簇间不相似性、以及这两个值最大值的比值。它可以看做簇间分离程度的度量。
评估它的相似性
$$s_i = \frac {b_i - a_i}{max(a_i, b_i)}$$
$s_i$是观测样本$i$的**轮廓系数**(Silhouette Coefficient), $a_i是i和同类的所有观测值之间的平均距离, a_i越小，说明样本i越应该被聚类到该簇,将a_i 称为样本i的簇内不相似度; 而b_i是i和不同类的所有观测的平均距离的最小值, 称为样本i的最近的簇间不相似性, b_i越大说明样本i越不属于其他簇$

判断：
- si接近1，则说明样本i聚类合理；

- si接近-1，则说明样本i更应该分类到另外的簇；

- 若si 近似为0，则说明样本i在两个簇的边界上。

In [ ]:
from sklearn import metrics
silhouette_samples = metrics.silhouette_samples(blobs, kmean.labels_)
silhouette_samples

In [ ]:
np.column_stack((classes[:5], silhouette_samples[:5]))

In [ ]:
# 数据点到形心的距离分布
plt.hist(silhouette_samples)
# 通常接近 1 的系数越高，分数就越高

In [ ]:
# 轮廓系数的均值通常用于描述整个模型的拟合度
silhouette_samples.mean()

In [ ]:
# 直接获取 轮廓系数的均值
metrics.silhouette_score(blobs, kmean.labels_)

In [ ]:
blobs, classes = make_blobs(500, centers=10)
sillhouette_avgs = []
for k in range(2, 60):
    kmean = KMeans(n_clusters=k).fit(blobs)
    sillhouette_avgs.append(metrics.silhouette_score(blobs, kmean.labels_))
plt.plot(sillhouette_avgs)

轮廓均值随着形心数量的变化情况. 最优的数量和实际的数量不一致, 这就是聚类的实际情况，十分普遍，我们不能获得正确的簇数量，我们只能估计簇数量的近似值

### 评估聚类的正确性
---
簇已知条件下的 KMeans 评估

In [ ]:
blobs, classes = make_blobs(500, centers=3)
kmean = KMeans(n_clusters=3).fit(blobs)

In [ ]:
colors = ['r', 'g', 'b']
for i in range(3):
    p = blobs[classes==i]
    plt.scatter(p[:, 0], p[:, 1], label=f'Cluster {i}', color=colors[i])
plt.legend()
plt.scatter(kmean.cluster_centers_[:, 0],
               kmean.cluster_centers_[:, 1], s=100,
               color='black',
               label='Centers') 
plt.title("Cluster With Ground Truth") 
plt.savefig('2020-1-11_1')

In [ ]:
kmean.labels_

In [ ]:
classes

In [ ]:
# 原始classes 变换类别编号
labels = kmean.labels_.copy()
labels[kmean.labels_==2] = 1  # 2->0
labels[kmean.labels_==0] = 2  # 0->1
labels[kmean.labels_==1] = 0  # 1->2
labels

In [ ]:
# 聚类的结果
for i in range(3):
    p = blobs[labels==i]
    plt.scatter(p[:, 0], p[:, 1], label=f'Cluster {i}', color=colors[i])

**准确度**

In [ ]:
# 各类别的准确度
for i in range(3):
    print((classes == labels)[classes==i].astype(int).mean())

第二个相似性度量是**互信息**（ mutual information score）得分。

$$I(X;Y)=\sum_x\sum_y p(x,y) log \frac{p(x,y)}{p(x)p(y)}$$
归一化的互信息(Normalized Mutual Information, NMI)
$$NMI(X;Y) = 2\frac {I(X;Y)}{H(X)+H(Y)}$$
分数靠近 0，就说明标签的分配可能不是按照相似过程生成的。但是分数靠近 1，就说明两个标签有很强的一致性。

In [ ]:
metrics.normalized_mutual_info_score(classes, kmean.labels_)

**惯性**（inertia）度量. 
惯性是每个数据点和它所分配的簇的平方差之和

In [ ]:
kmean.inertia_

### k-means 聚类手写数字
---

In [ ]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale

In [ ]:
digits = load_digits()
data = scale(digits.data)  # z = (x-u) / s  归一化的data
digits.images[0], digits.target[0]

In [ ]:
# 打印图片信息
def print_digits(images, y, max_n=10):
    fig = plt.figure(figsize=(12, 12))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1,
                       hspace=0.05, wspace=0.005)
    i = 0
    while i < max_n and i < images.shape[0]:
        p = fig.add_subplot(20, 20, i+1, xticks=[], yticks=[])
        p.imshow(images[i], cmap=plt.cm.bone)
        p.text(0, 14, str(y[i]))
        i += 1

In [ ]:
# 前10张图片
print_digits(digits.images, digits.target)

In [ ]:
plt.imshow(img)

In [ ]:
from sklearn.model_selection import train_test_split

# 训练集 测试集 
X_train, X_test, y_train, y_test, images_train, images_test = train_test_split(
    data, digits.target, digits.images, test_size=0.3, random_state=0)

In [ ]:
n_digits = len(np.unique(y_train))
labels = y_train
n_digits

In [ ]:
from sklearn.cluster import KMeans

clf = KMeans(init='k-means++', n_clusters=10, random_state=42)
clf.fit(X_train)

In [ ]:
# 图像 预测分类
print_digits(images_train, clf.labels_, max_n=10)
# 预测的类编号与实际图片的数值无关, 是clf自身的分类

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
# 显示每个簇的10个图像
def print_cluster(images, y_pred, cluster_number):
    images = images[y_pred == cluster_number]
    y_pred = y_pred[y_pred == cluster_number]
    print_digits(images, y_pred, max_n=10)
for i in range(10):
    print_cluster(images_test, y_pred, i)

可以看到有些簇非常清晰, 如簇2对应数字0, 簇7对应数字6;有些簇不是那么清楚, 如簇6中有9、8和3

---
#### 评估
我们如何评估我们的表现？精确率和所有这些东西都不起作用，因为我们没有可比较的目标类。为了评估，我们需要知道“真正的”簇，无论这意味着什么。我们可以假设，对于我们的示例，每个簇包括特定数字的每个绘图，并且仅包括该数字。知道这一点，我们可以计算我们的簇分布和预期之间的调整**兰德系数**。兰德系数是一个类似的准确率量度，但它考虑到两个分布中的类可以有不同名称的事实。也就是说，如果我们更改类名，索引不会改变。调整兰德系数试图消除偶然发生的结果的巧合。当两个集合中具有完全相同的簇时，兰德系数等于 1，而当没有簇共享数据点时，它等于零。

In [ ]:
from sklearn import metrics
# adjusted rand socre
metrics.adjusted_rand_score(y_test, y_pred)

In [ ]:
# 混淆矩阵
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
# 数字0全部分类到了簇2, 完全一致
confusion_matrix[0]

In [ ]:
# 数字9 5个分到簇0, 50个在簇6, 2个簇8
confusion_matrix[-1]

### 使用 KMeans 聚类来量化图像
---

In [ ]:
img = plt.imread('01.png')
plt.imshow(img)

In [ ]:
img.shape

为了实际量化图像，我们需要将其转换为二维数组，长为1080x1920，宽为 RGB 值. 

思考它的更好的方法，是拥有一堆三维空间中的数据点，并且对点进行聚类来降低图像中的不同颜色的数量 -- 这是一个简单的量化方式。

In [ ]:
x, y, z = img.shape
long_img = img.reshape(x*y, z)
long_img.shape

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)  # 5中颜色
kmeans.fit(long_img)

In [ ]:
centers = kmeans.cluster_centers_ 
centers

In [ ]:
labels = kmeans.labels_
labels[:5]

In [ ]:
plt.imshow(centers[labels].reshape(x, y, z))

### MiniBatchKMeans 小批量KMeans聚类
---
MiniBatchKMeans 是 KMeans 算法的一个变种，它使用小批量(mini-batches)来减少计算时间，而这多个批次仍然尝试优化相同的目标函数。小批量是输入数据的子集，在每次训练迭代中随机抽样。这些小批量大大减少了收敛到局部解所需的计算量。 与其他降低 k-means 收敛时间的算法不同，小批量 k-means 产生的结果通常只比标准算法略差。

该算法在两个步骤之间进行迭代，类似于 vanilla k-means 。在第一步， b 样本是从数据集中随机抽取的，形成一个小批量。然后将它们分配到最近的质心。 在第二步，质心被更新。与 k-means 不同, 该变种算法是基于每个样本(per-sample)。对于小批量中的每个样本，通过取样本的流平均值(streaming average)和分配给该质心的所有先前样本来更新分配的质心。 这具有随时间降低质心的变化率（rate of change）的效果。执行这些步骤直到达到收敛或达到预定次数的迭代。

MiniBatchKMeans 收敛速度比 KMeans快 ，但是结果的质量会降低

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import scale
from sklearn.cluster import MiniBatchKMeans

iris = load_iris()
X, y = iris.data, iris.target

X_std = scale(X)

In [ ]:
#batch_size控制每批中随机选择的观测数。 批量越大，训练过程的计算成本就越高。
clf = MiniBatchKMeans(n_clusters=3, random_state=0, batch_size=100)
clf.fit(X_std)

In [ ]:
clf.labels_

### MiniBatchKMeans和KMeans的对比


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, MiniBatchKMeans

blobs, labels = make_blobs(int(1e6), 3)
kmeans = KMeans(n_clusters=3)
minibatch = MiniBatchKMeans(n_clusters=3)

In [ ]:
# 对比运行时间
%time kmeans.fit(blobs)

In [ ]:
%time minibatch.fit(blobs)

In [ ]:
# 类中心
kmeans.cluster_centers_

In [ ]:
minibatch.cluster_centers_

In [ ]:
kmeans.cluster_centers_[[0]]

In [ ]:
from sklearn.metrics import pairwise

pairwise.pairwise_distances(kmeans.cluster_centers_[[0]], minibatch.cluster_centers_[[0]])

5.6.23 Comparison of the K-Means and MiniBatchKMeans clustering algorithms

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets.samples_generator import make_blobs
# #############################################################################
# Generate sample data
np.random.seed(0)
batch_size = 45
centers = [[1, 1], [-1, -1], [1, -1]]
n_clusters = len(centers)
X, labels_true = make_blobs(n_samples=3000, centers=centers, cluster_std=0.7)
# #############################################################################
# Compute clustering with Means
k_means = KMeans(init='k-means++', n_clusters=3, n_init=10)
t0 = time.time()
k_means.fit(X)
t_batch = time.time() - t0
# #############################################################################
# Compute clustering with MiniBatchKMeans
mbk = MiniBatchKMeans(init='k-means++', n_clusters=3, batch_size=batch_size,
n_init=10, max_no_improvement=10, verbose=0)
t0 = time.time()

mbk.fit(X)
t_mini_batch = time.time() - t0
# #############################################################################
# Plot result
fig = plt.figure(figsize=(8, 3))
fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
colors = ['#4EACC5', '#FF9C34', '#4E9A06']
# We want to have the same colors for the same cluster from the
# MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per
# closest one.
k_means_cluster_centers = np.sort(k_means.cluster_centers_, axis=0)
mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis=0)
k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)
mbk_means_labels = pairwise_distances_argmin(X, mbk_means_cluster_centers)
order = pairwise_distances_argmin(k_means_cluster_centers,
mbk_means_cluster_centers)
# KMeans
ax = fig.add_subplot(1, 3, 1)
for k, col in zip(range(n_clusters), colors):
    my_members = k_means_labels == k
    cluster_center = k_means_cluster_centers[k]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',
    markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
    markeredgecolor='k', markersize=6)
ax.set_title('KMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8, 'train time: %.2fs\ninertia: %f' % (
    t_batch, k_means.inertia_))

# MiniBatchKMeans
ax = fig.add_subplot(1, 3, 2)
for k, col in zip(range(n_clusters), colors):
    my_members = mbk_means_labels == order[k]
    cluster_center = mbk_means_cluster_centers[order[k]]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',
    markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
    markeredgecolor='k', markersize=6)
ax.set_title('MiniBatchKMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8, 'train time: %.2fs\ninertia: %f' %
    (t_mini_batch, mbk.inertia_))
# Initialise the different array to all False
different = (mbk_means_labels == 4)
ax = fig.add_subplot(1, 3, 3)
for k in range(n_clusters):
    different += ((k_means_labels == k) != (mbk_means_labels == order[k]))
identic = np.logical_not(different)
ax.plot(X[identic, 0], X[identic, 1], 'w',
markerfacecolor='#bbbbbb', marker='.')
ax.plot(X[different, 0], X[different, 1], 'w',
markerfacecolor='m', marker='.')
ax.set_title('Difference')
ax.set_xticks(())
ax.set_yticks(())
plt.show()